# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298447172463                   -0.85    5.6
  2   -8.300204581906       -2.76       -1.25    1.0
  3   -8.300438972089       -3.63       -1.89    1.0
  4   -8.300460718678       -4.66       -2.71    3.4
  5   -8.300463681344       -5.53       -3.08    4.8
  6   -8.300464111197       -6.37       -3.22    2.6
  7   -8.300464353418       -6.62       -3.34    1.4
  8   -8.300464507856       -6.81       -3.49    1.4
  9   -8.300464627473       -6.92       -3.78    4.2
 10   -8.300464638770       -7.95       -3.96    1.2
 11   -8.300464641914       -8.50       -4.23    1.8


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -16.66098444963                   -0.70    6.0
  2   -16.67854572873       -1.76       -1.14    3.4
  3   -16.67920765307       -3.18       -1.84    2.1
  4   -16.67927774475       -4.15       -2.71    2.1
  5   -16.67928603427       -5.08       -3.12    4.0
  6   -16.67928619803       -6.79       -3.49    2.1
  7   -16.67928621963       -7.67       -3.96    1.6
  8   -16.67928622395       -8.36       -4.51    2.5


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.32565243497                   -0.56    6.6
  2   -33.33276370473       -2.15       -1.00    1.0
  3   -33.33411403658       -2.87       -1.74    4.4
  4   -33.33426141266       -3.83       -2.63    3.8
  5   -33.33512963123       -3.06       -2.33    8.0
  6   -33.33694246967       -2.74       -2.51    5.0
  7   -33.33694237153   +   -7.01       -2.52    1.1
  8   -33.33689413171   +   -4.32       -2.45    2.8
  9   -33.33686065197   +   -4.48       -2.34    1.1
 10   -33.33685070522   +   -5.00       -2.32    1.0
 11   -33.33684870210   +   -5.70       -2.32    1.0
 12   -33.33685163207       -5.53       -2.33    1.0
 13   -33.33686040133       -5.06       -2.35    1.0
 14   -33.33686364912       -5.49       -2.36    

When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -8.298552209672                   -0.85    5.9
  2   -8.300272328318       -2.76       -1.59    1.0
  3   -8.300434263660       -3.79       -2.56    1.9
  4   -8.300388156659   +   -4.34       -2.31    4.6
  5   -8.300464261896       -4.12       -3.48    1.0
  6   -8.300464601460       -6.47       -3.92    5.2
  7   -8.300464637464       -7.44       -4.20    2.4


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -33.28773647770                   -0.56    6.8
  2   -33.24906031436   +   -1.41       -1.23    1.4
  3   +25.71518317984   +    1.77       -0.18   10.0
  4   -33.29511254622        1.77       -1.50    6.0
  5   -32.45395382139   +   -0.08       -1.05    4.1
  6   -30.82033654053   +    0.21       -0.86    4.5
  7   -33.31174380542        0.40       -1.78    3.8
  8   -33.33304906018       -1.67       -2.01    2.6
  9   -33.33426476280       -2.92       -2.09    2.2
 10   -33.33645304024       -2.66       -2.46    2.0
 11   -33.33686178348       -3.39       -2.84    2.1
 12   -33.33690364393       -4.38       -3.02    3.2
 13   -33.33692993996       -4.58       -3.32    3.4
 14   -33.33694246883       -4.90       -3.75    

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.